In [1]:
import pandas as pd
import numpy as np
import os, os.path
import pickle

In [2]:
SRR2920492 = pd.read_table("../../../data/Corces2016_bulk_ATAC/01_ENCODE_cCRE_coverages/Bcell/SRR2920492_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed", sep='\t', header=None)

### 6th column refers to coverage

In [3]:
SRR2920492

,0,1,2,3,4,5,6,7,8,9
0,chr1,181251,181601,EH38D2115304,EH38E1310153,"pELS,CTCF-bound",6,128,350,0.365714
1,chr1,190865,191071,EH38D2115305,EH38E1310154,"dELS,CTCF-bound",0,0,206,0.000000
2,chr1,778562,778912,EH38D2115324,EH38E1310158,"PLS,CTCF-bound",59,346,350,0.988571
3,chr1,779086,779355,EH38D2115325,EH38E1310159,"PLS,CTCF-bound",21,206,269,0.765799
4,chr1,779727,780060,EH38D2115326,EH38E1310160,"pELS,CTCF-bound",2,152,333,0.456456
...,...,...,...,...,...,...,...,...,...,...
926530,chrY,56842374,56842545,EH38D4327456,EH38E2776491,"dELS,CTCF-bound",6,134,171,0.783626
926531,chrY,56844431,56844674,EH38D4327461,EH38E2776496,"dELS,CTCF-bound",3,73,243,0.300411
926532,chrY,56857410,56857680,EH38D4327481,EH38E2776512,"CTCF-only,CTCF-bound",12,181,270,0.670370
926533,chrY,56857917,56858119,EH38D4327482,EH38E2776513,"CTCF-only,CTCF-bound",10,183,202,0.905941


In [4]:
SRR2920492['merged_peak_name']=SRR2920492[0].astype(str)+'_'+SRR2920492[1].astype(str)+'_'+SRR2920492[2].astype(str)

SRR2920492.index = SRR2920492.loc[:,"merged_peak_name"]
SRR2920492 = pd.DataFrame(SRR2920492.loc[:,6])
SRR2920492

,6
merged_peak_name,
chr1_181251_181601,6
chr1_190865_191071,0
chr1_778562_778912,59
chr1_779086_779355,21
chr1_779727_780060,2
...,...
chrY_56842374_56842545,6
chrY_56844431_56844674,3
chrY_56857410_56857680,12


## columns --> number of bulk data

In [5]:
corces_cell_types = ["Bcell","CD34_Bone_Marrow","CD34_Cord_Blood","CD4Tcell","CD8Tcell","CLP","CMP","Ery","GMP","HSC","LMPP","MEP","MPP","Mono","NKcell"]
path_dir_base = "../../../data/Corces2016_bulk_ATAC/01_ENCODE_cCRE_coverages/"
ENCODE_cCRE_overlapped_bedFiles_dir=[]
SRR_IDs=[]
# convert to a pandas df for the easy access to incex etc
for cell_type in corces_cell_types:
    path_dir = path_dir_base + cell_type
    for bed_file in os.listdir(path_dir):
        if bed_file.endswith("_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed"):
            #print(f)
            ENCODE_cCRE_overlapped_bedFiles_dir.append(bed_file)
            SRR_IDs.append(bed_file.split('_')[0])

In [6]:
# number of bulk atac data --> number of columns
num_ENCODE_cCRE_overlapped_bedFiles =  len(ENCODE_cCRE_overlapped_bedFiles_dir)
num_ENCODE_cCRE_overlapped_bedFiles

80

In [7]:
# features (peaks)
ENCODE_cCREs = pd.read_table("../../../data/ENCODE_cCREs/ENCOCE_cREs_merged_names.tsv", sep='\t', header=None)


In [8]:
ENCODE_cCREs.shape

(926535, 1)

## create bulk_by_ENCODE_cCREs matrix

In [9]:
corces_cell_types = ["Bcell","CD34_Bone_Marrow","CD34_Cord_Blood","CD4Tcell","CD8Tcell","CLP","CMP","Ery","GMP","HSC","LMPP","MEP","MPP","Mono","NKcell"]
path_dir_base = "../../../data/Corces2016_bulk_ATAC/01_ENCODE_cCRE_coverages/"
bulk_by_ENCODE_peaks = np.zeros((len(ENCODE_cCREs),0))
# convert to a pandas df for the easy access to incex etc
bulk_by_ENCODE_peaks_df = pd.DataFrame(bulk_by_ENCODE_peaks,index=ENCODE_cCREs[0])
for cell_type in corces_cell_types:
    path_dir = path_dir_base + cell_type
    for bed_file in os.listdir(path_dir):
        if bed_file.endswith("_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed"):
            print(bed_file)
            SRR_name = bed_file.split('_')[0]
            full_path = path_dir +"/"+ bed_file
            SRR_df = pd.read_table(full_path, sep='\t', header=None)
            SRR_df['merged_peak_name']=SRR_df[0].astype(str)+'_'+SRR_df[1].astype(str)+'_'+SRR_df[2].astype(str)
            SRR_df.index = SRR_df.loc[:,"merged_peak_name"]
            SRR_df = pd.DataFrame(SRR_df.loc[:,6])
            SRR_df.columns = [SRR_name]
            bulk_by_ENCODE_peaks_df= pd.concat([bulk_by_ENCODE_peaks_df,SRR_df], axis=1)

SRR2920492_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920513_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920517_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920544_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920489_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920490_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920491_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920493_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920496_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920514_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920518_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920519_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920494_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920497_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920515_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920520_NOTmerged_scATAC_overlap_ENCODE_cCRE_coverage.bed
SRR2920521_NOTmerged_scA

In [10]:
bulk_by_ENCODE_peaks_df

,SRR2920492,SRR2920513,SRR2920517,SRR2920544,SRR2920489,SRR2920490,SRR2920491,SRR2920493,SRR2920496,SRR2920514,...,SRR2920487,SRR2920488,SRR2920542,SRR2920543,SRR2920495,SRR2920511,SRR2920512,SRR2920516,SRR2920526,SRR2920527
chr1_181251_181601,6,9,9,0,0,2,2,4,6,6,...,2,0,3,8,3,3,4,17,2,6
chr1_190865_191071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
chr1_778562_778912,59,319,518,183,183,198,220,210,271,242,...,64,83,204,155,163,146,115,529,262,322
chr1_779086_779355,21,40,85,35,35,23,45,33,42,42,...,4,6,23,25,40,22,21,95,34,36
chr1_779727_780060,2,13,8,2,5,0,5,1,9,6,...,4,4,5,1,5,2,3,8,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrY_56842374_56842545,6,16,10,0,2,0,5,0,7,7,...,1,1,9,13,5,12,3,11,6,8
chrY_56844431_56844674,3,18,5,0,0,0,1,0,5,3,...,0,0,1,5,2,6,8,8,3,2
chrY_56857410_56857680,12,19,0,0,4,0,1,5,4,8,...,7,2,8,7,9,5,6,6,3,2
chrY_56857917_56858119,10,35,4,0,7,0,8,0,5,10,...,5,2,6,0,7,12,5,14,9,0


## SRR cell type mapping

In [11]:
metadata = pd.read_csv("../../../data/Corces2016_bulk_ATAC/GSE74912_metadata.csv")

### filter only for the healthy tissues

In [12]:
metadata = metadata[metadata["sample_type"] == "primary human blood cell from fresh sample"]

In [13]:
metadata

,Run,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Cell_type,Center Name,Consent,...,LibraryLayout,LibrarySelection,LibrarySource,Organism,Platform,ReleaseDate,Sample Name,sample_type,source_name,SRA Study
0,SRR2920466,OTHER,152,1009532776,PRJNA301969,SAMN04262106,402134906,hematopoietic stem cell,GEO,public,...,PAIRED,other,GENOMIC,Homo sapiens,ILLUMINA,2016-06-01T00:00:00Z,GSM1937376,primary human blood cell from fresh sample,HSC,SRP066100
1,SRR2920467,OTHER,152,1020967888,PRJNA301969,SAMN04262107,406010586,multipotent progenitor cell,GEO,public,...,PAIRED,other,GENOMIC,Homo sapiens,ILLUMINA,2016-06-01T00:00:00Z,GSM1937377,primary human blood cell from fresh sample,MPP,SRP066100
2,SRR2920468,OTHER,152,1182654544,PRJNA301969,SAMN04262108,466911262,lymphoid-primed multipotent progenitor cell,GEO,public,...,PAIRED,other,GENOMIC,Homo sapiens,ILLUMINA,2016-06-01T00:00:00Z,GSM1937378,primary human blood cell from fresh sample,LMPP,SRP066100
3,SRR2920469,OTHER,152,1257813224,PRJNA301969,SAMN04262109,498852967,common myeloid progenitor cell,GEO,public,...,PAIRED,other,GENOMIC,Homo sapiens,ILLUMINA,2016-06-01T00:00:00Z,GSM1937379,primary human blood cell from fresh sample,CMP,SRP066100
4,SRR2920470,OTHER,152,1455210456,PRJNA301969,SAMN04262110,571808638,common myeloid progenitor cell,GEO,public,...,PAIRED,other,GENOMIC,Homo sapiens,ILLUMINA,2016-06-01T00:00:00Z,GSM1937380,primary human blood cell from fresh sample,CMP,SRP066100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,SRR2920541,OTHER,152,4068462400,PRJNA301969,SAMN04262123,2017246584,megakaryocyte erythroid progenitor cell,GEO,public,...,PAIRED,other,GENOMIC,Homo sapiens,ILLUMINA,2016-06-01T00:00:00Z,GSM1937451,primary human blood cell from fresh sample,MEP,SRP066100
76,SRR2920542,OTHER,152,5824919376,PRJNA301969,SAMN04262124,2888818006,CD14+ monocyte cell,GEO,public,...,PAIRED,other,GENOMIC,Homo sapiens,ILLUMINA,2016-06-01T00:00:00Z,GSM1937452,primary human blood cell from fresh sample,Mono,SRP066100
77,SRR2920543,OTHER,152,7468277560,PRJNA301969,SAMN04262125,3655086815,CD14+ monocyte cell,GEO,public,...,PAIRED,other,GENOMIC,Homo sapiens,ILLUMINA,2016-06-01T00:00:00Z,GSM1937453,primary human blood cell from fresh sample,Mono,SRP066100
78,SRR2920544,OTHER,152,2087880512,PRJNA301969,SAMN04262070,1176993517,CD19+CD20+ B cell,GEO,public,...,PAIRED,other,GENOMIC,Homo sapiens,ILLUMINA,2016-06-01T00:00:00Z,GSM1937454,primary human blood cell from fresh sample,Bcell,SRP066100


In [14]:
metadata.columns

Index(['Run', 'Assay Type', 'AvgSpotLen', 'Bases', 'BioProject', 'BioSample',
       'Bytes', 'Cell_type', 'Center Name', 'Consent', 'DATASTORE filetype',
       'DATASTORE provider', 'DATASTORE region', 'donorid', 'Experiment',
       'GEO_Accession (exp)', 'Instrument', 'LibraryLayout',
       'LibrarySelection', 'LibrarySource', 'Organism', 'Platform',
       'ReleaseDate', 'Sample Name', 'sample_type', 'source_name',
       'SRA Study'],
      dtype='object')

In [15]:
SRR_celltype_dict = dict()
for cell_type in sorted(set(metadata["source_name"])):
    # check if there are technical replicates (if there are multiple samples with same bioSample ID)
    sub_ = metadata[metadata.source_name==cell_type]
    if len(set(sub_.BioSample)) != sub_.shape[0]:
        print("yes")
        print(cell_type)
    else:
        print(cell_type)
        #print(sub_['donorid'])
        for srr_idx in range(sub_.shape[0]): 
            SRR_celltype_dict[sub_.iloc[srr_idx][0]] = cell_type.replace(" ", "_") + "_"+ str(srr_idx+1) 
            #print(sub_.iloc[srr_idx][0])
            #print(cell_type + "_"+ str(srr_idx+1))
            # begin0 = "\"atac.fastqs_" 
            # begin1  = "rep" + str(srr_idx+1)
            # begin2_R1  = "_R1\" : [ "
            # begin2_R2  = "_R2\" : [ "
            # code_R1 =  "\""  +fastq_path +sub_.iloc[srr_idx][0] + "_1.fastq\""
            # code_R2 =  "\""  +fastq_path +sub_.iloc[srr_idx][0] + "_2.fastq\"" 
            # end = " ]"
            # combined_R1 = "    "+ begin0 + begin1 + begin2_R1 + code_R1 +end + ", "
            # combined_R2 = "    "+begin0 + begin1 + begin2_R2 + code_R2 +end + ", "
            # print(combined_R1)
            # print(combined_R2)

            

Bcell
CD34 Bone Marrow
CD34 Cord Blood
CD4Tcell
CD8Tcell
CLP
CMP
Ery
GMP
HSC
LMPP
MEP
MPP
Mono
NKcell


In [16]:
SRR_celltype_dict

{'SRR2920492': 'Bcell_1',
 'SRR2920513': 'Bcell_2',
 'SRR2920517': 'Bcell_3',
 'SRR2920544': 'Bcell_4',
 'SRR2920489': 'CD34_Bone_Marrow_1',
 'SRR2920490': 'CD34_Bone_Marrow_2',
 'SRR2920491': 'CD34_Cord_Blood_1',
 'SRR2920493': 'CD4Tcell_1',
 'SRR2920496': 'CD4Tcell_2',
 'SRR2920514': 'CD4Tcell_3',
 'SRR2920518': 'CD4Tcell_4',
 'SRR2920519': 'CD4Tcell_5',
 'SRR2920494': 'CD8Tcell_1',
 'SRR2920497': 'CD8Tcell_2',
 'SRR2920515': 'CD8Tcell_3',
 'SRR2920520': 'CD8Tcell_4',
 'SRR2920521': 'CD8Tcell_5',
 'SRR2920498': 'CLP_1',
 'SRR2920499': 'CLP_2',
 'SRR2920522': 'CLP_3',
 'SRR2920528': 'CLP_4',
 'SRR2920545': 'CLP_5',
 'SRR2920469': 'CMP_1',
 'SRR2920470': 'CMP_2',
 'SRR2920481': 'CMP_3',
 'SRR2920482': 'CMP_4',
 'SRR2920500': 'CMP_5',
 'SRR2920501': 'CMP_6',
 'SRR2920536': 'CMP_7',
 'SRR2920537': 'CMP_8',
 'SRR2920502': 'Ery_1',
 'SRR2920503': 'Ery_2',
 'SRR2920504': 'Ery_3',
 'SRR2920523': 'Ery_4',
 'SRR2920524': 'Ery_5',
 'SRR2920525': 'Ery_6',
 'SRR2920529': 'Ery_7',
 'SRR2920530': '

In [17]:
bulk_by_ENCODE_peaks_df_annotated = bulk_by_ENCODE_peaks_df.copy()
bulk_by_ENCODE_peaks_df_annotated.rename(columns = SRR_celltype_dict, inplace=True)
bulk_by_ENCODE_peaks_df_annotated

,Bcell_1,Bcell_2,Bcell_3,Bcell_4,CD34_Bone_Marrow_1,CD34_Bone_Marrow_2,CD34_Cord_Blood_1,CD4Tcell_1,CD4Tcell_2,CD4Tcell_3,...,Mono_3,Mono_4,Mono_5,Mono_6,NKcell_1,NKcell_2,NKcell_3,NKcell_4,NKcell_5,NKcell_6
chr1_181251_181601,6,9,9,0,0,2,2,4,6,6,...,2,0,3,8,3,3,4,17,2,6
chr1_190865_191071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
chr1_778562_778912,59,319,518,183,183,198,220,210,271,242,...,64,83,204,155,163,146,115,529,262,322
chr1_779086_779355,21,40,85,35,35,23,45,33,42,42,...,4,6,23,25,40,22,21,95,34,36
chr1_779727_780060,2,13,8,2,5,0,5,1,9,6,...,4,4,5,1,5,2,3,8,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrY_56842374_56842545,6,16,10,0,2,0,5,0,7,7,...,1,1,9,13,5,12,3,11,6,8
chrY_56844431_56844674,3,18,5,0,0,0,1,0,5,3,...,0,0,1,5,2,6,8,8,3,2
chrY_56857410_56857680,12,19,0,0,4,0,1,5,4,8,...,7,2,8,7,9,5,6,6,3,2
chrY_56857917_56858119,10,35,4,0,7,0,8,0,5,10,...,5,2,6,0,7,12,5,14,9,0


In [18]:
bulk_by_ENCODE_peaks_df

,SRR2920492,SRR2920513,SRR2920517,SRR2920544,SRR2920489,SRR2920490,SRR2920491,SRR2920493,SRR2920496,SRR2920514,...,SRR2920487,SRR2920488,SRR2920542,SRR2920543,SRR2920495,SRR2920511,SRR2920512,SRR2920516,SRR2920526,SRR2920527
chr1_181251_181601,6,9,9,0,0,2,2,4,6,6,...,2,0,3,8,3,3,4,17,2,6
chr1_190865_191071,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
chr1_778562_778912,59,319,518,183,183,198,220,210,271,242,...,64,83,204,155,163,146,115,529,262,322
chr1_779086_779355,21,40,85,35,35,23,45,33,42,42,...,4,6,23,25,40,22,21,95,34,36
chr1_779727_780060,2,13,8,2,5,0,5,1,9,6,...,4,4,5,1,5,2,3,8,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrY_56842374_56842545,6,16,10,0,2,0,5,0,7,7,...,1,1,9,13,5,12,3,11,6,8
chrY_56844431_56844674,3,18,5,0,0,0,1,0,5,3,...,0,0,1,5,2,6,8,8,3,2
chrY_56857410_56857680,12,19,0,0,4,0,1,5,4,8,...,7,2,8,7,9,5,6,6,3,2
chrY_56857917_56858119,10,35,4,0,7,0,8,0,5,10,...,5,2,6,0,7,12,5,14,9,0


In [19]:
# Saving the objects:
with open('../../../data/Corces2016_bulk_ATAC/02_ENCODE_coverage_by_prototypes_matrix/Corces2016bulkATAC_ENCODE_cCRE_overlappingPeaks_annotated.pkl', 'wb') as f:
    pickle.dump(bulk_by_ENCODE_peaks_df_annotated, f)

In [20]:
#Saving the objects:
with open('../../../data/Corces2016_bulk_ATAC/02_ENCODE_coverage_by_prototypes_matrix/Corces2016bulkATAC_ENCODE_cCRE_overlappingPeaks.pkl', 'wb') as f:
    pickle.dump(bulk_by_ENCODE_peaks_df, f)



In [21]:
import os
cwd = os.getcwd()
cwd

'/project/scATAC_analysis/scATAcat_notebooks_for_paper/jan2024/preprocessing/Corces2016_bulk_ATAC/ENCODE_ccRE_coverages'